In [7]:
import requests
import io
import sys

from bs4 import BeautifulSoup
import pandas as pd

stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

In [33]:
url=("http://numismatics.org/ocre/query.csv?start=")
    
batch = url + "0"
rawContent = requests.get(batch).content
raw_data = pd.read_csv(io.StringIO(rawContent.decode('utf-8')))

In [34]:
for i in range(1, 63):
    i *= 1000
    batch = url + str(i)
    rawContent = requests.get(batch).content
    df = pd.read_csv(io.StringIO(rawContent.decode('utf-8')))
    raw_data = raw_data.append(df, ignore_index=True)

In [35]:
raw_data.tail()

,URI,Title,RecordId,Authority,Degree,Deity,Denomination,Dynasty,Engraver,Era,...,Obverse Legend,Obverse Type,Object Type,Portrait,Reference,Region,Reverse Legend,Reverse Type,Year,Date Record Modified
42743,http://numismatics.org/ocre/id/ric.5.gall(1).5,RIC V Gallienus (joint reign) 5,ric.5.gall(1).5,Valerian|Gallienus,NaN,Mars,Antoninianus,NaN,NaN,NaN,...,GALLIENVS P F AVG,"Bust of Gallienus, radiate, cuirassed, right",Coin,Gallienus,NaN,Gallia,P M TR P V COS III,"Mars, walking right, holding spear in right ha...",257,2018-08-08T16:20:41Z
42744,http://numismatics.org/ocre/id/ric.5.gall(1).4,RIC V Gallienus (joint reign) 4,ric.5.gall(1).4,Valerian|Gallienus,NaN,Victory,Aureus,NaN,NaN,NaN,...,GALLIENVS P F AVG,"Head of Gallienus, laureate, right",Coin,Gallienus,NaN,Gallia,VICTORIAE AVGG,"Victory, winged, draped, standing in biga",258|259,2018-08-08T16:20:41Z
42745,http://numismatics.org/ocre/id/ric.5.gall(1).163,RIC V Gallienus (joint reign) 163,ric.5.gall(1).163,Valerian|Gallienus,NaN,Providentia,Antoninianus,NaN,NaN,NaN,...,IMP GALLIENVS P F AVG GERM,"Head of Gallienus, radiate, right",Coin,Gallienus,NaN,Italy,PROVIDENTIA AVGG,"Providentia, draped, standing left, leaning on...",257|258,2018-08-08T16:21:11Z
42746,http://numismatics.org/ocre/id/ric.5.gall(1).160,RIC V Gallienus (joint reign) 160,ric.5.gall(1).160,Valerian|Gallienus,NaN,Providentia,Antoninianus,NaN,NaN,NaN,...,IMP GALLIENVS P F AVG GERM,"Head of Gallienus, radiate, right",Coin,Gallienus,NaN,Italy,PROVIDENTIA AVGG,"Providentia, draped, standing left, holding ba...",257|258,2018-08-08T16:21:11Z
42747,http://numismatics.org/ocre/id/ric.5.gall(1).147,RIC V Gallienus (joint reign) 147,ric.5.gall(1).147,Valerian|Gallienus,NaN,Liberalitas,Antoninianus,NaN,NaN,NaN,...,IMP C P LIC GALLIENVS AVG,"Bust of Gallienus, radiate, cuirassed, right",Coin,Gallienus,NaN,Italy,LIBERALITAS AVGG,"Liberalitas, draped, standing left, holding te...",254|255,2018-08-08T16:21:08Z


In [30]:
data = None
for url in links: #data['URI'].values:
    html_data = requests.get(url).content
    soup = BeautifulSoup(html_data, 'html.parser')
    csv_url_container = soup.find('a', {'title': 'Download CSV'})
    if csv_url_container is None:
        continue
    csv_url = csv_url_container['href']
    items = pd.read_csv(csv_url)
    filtered_items = items[(items['collection'] != 'American Numismatic Society') & 
                           (items['collection'] != 'British Museum')]
    if data:
        data = data.append(filtered_items, ignore_index=True)
    else:
        data = filtered_items.copy()

In [29]:
collections = ['American Numismatic Society', 'British Museum']
items[(items['collection'] != 'American Numismatic Society') & (items['collection'] != 'British Museum')]

,object,title,identifier,findUri,findspot,hoard,collection,publisher,dataset,datasetTitle,...,obvThumb,revThumb,obvRef,revRef,comThumb,comRef,obvManifest,revManifest,comManifest,model
1,http://ikmk.smb.museum/object?id=18207142,"Traianus: Restitution, 112-114 n. Chr.",DE-MUS-814819/18207142,NaN,NaN,NaN,Münzkabinett Berlin,Münzkabinett der Staatlichen Museen zu Berlin,http://ikmk.smb.museum/,Münzkabinett der Staatlichen Museen zu Berlin,...,http://ikmk.smb.museum/mk-edit/images/n7/7322/...,http://ikmk.smb.museum/mk-edit/images/n7/7322/...,http://ikmk.smb.museum/mk-edit/images/n7/7322/...,http://ikmk.smb.museum/mk-edit/images/n7/7322/...,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
raw_data[raw_data['URI'] == 'http://numismatics.org/ocre/id/ric.2.tr.797']

,URI,Title,RecordId,Authority,Degree,Deity,Denomination,Dynasty,Engraver,Era,...,Obverse Legend,Obverse Type,Object Type,Portrait,Reference,Region,Reverse Legend,Reverse Type,Year,Date Record Modified
28555,http://numismatics.org/ocre/id/ric.2.tr.797,RIC II Trajan 797,ric.2.tr.797,Trajan,NaN,Libertas,Denarius,NaN,NaN,NaN,...,LIBERTAS,"Head of Libertas, right",Coin,Brutus,NaN,Italy,IMP CAES TRAIAN AVG GER DAC P P REST BRVTVS,"Marcus Junius Brutus walking left, flanked by ...",98|117,2017-01-27T14:20:06.3Z


In [5]:
data.to_csv("../Data/OCRE.csv")